**Stage2 - validation**

In [ ]:
# @title 1. Environment and weight initialization
import os
import sys
import torch
import json
from PIL import Image
from google.colab import drive

# 1. Mount drive to access the finalized Stage 2 adapters.
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 2. Path configuration.
PROJECT_ROOT = '/content/drive/My Drive/projects/EarthShader'
STAGE2_PATH = os.path.join(PROJECT_ROOT, 'checkpoints/stage2_final')
BASE_MODEL_ID = "Qwen/Qwen2-VL-7B-Instruct"

# 3. Installation of essential inference tools.
print("Initializing verification environment...")
!pip install -q -U bitsandbytes
!pip install -q git+https://github.com/huggingface/transformers peft accelerate qwen-vl-utils

from transformers import Qwen2VLForConditionalGeneration, AutoProcessor, BitsAndBytesConfig
from peft import PeftModel

# 4. Model loading with 4-bit quantization for T4 efficiency.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print(f"Loading Stage 2 model from: {STAGE2_PATH}")
model = Qwen2VLForConditionalGeneration.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# Load the trained Stage 2 adapters.
model = PeftModel.from_pretrained(model, STAGE2_PATH)
model.eval()

processor = AutoProcessor.from_pretrained(BASE_MODEL_ID)

print("[SUCCESS] Dual-Stage verification engine is online.")

In [ ]:
# @title 2. Dual-Stage regression audit
def perform_inference(image_path, label="Test"):
    """Runs the model on a single image and returns the GLSL code."""
    image = Image.open(image_path).convert("RGB")

    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are an EarthShader SDF compiler. Translate the visual primitive into a valid GLSL mainImage function using the common.glsl library."}]
        },
        {
            "role": "user",
            "content": [{"type": "image"}, {"type": "text", "text": "Reverse engineer the GLSL shader code for this texture. Include analysis."}]
        }
    ]

    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[image], return_tensors="pt").to("cuda")

    with torch.no_grad():
        generated_ids = model.generate(**inputs, max_new_tokens=768)

    return processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

# 1. Stage 1 Spotcheck: Single primitives (Checking for Spatial Precision).
print("\n" + "="*60)
print("STAGE 1 REGRESSION CHECK (Spatial Precision)")
print("="*60)
s1_test_img = os.path.join(PROJECT_ROOT, 'dataset/stage1/images/prim_10005.png')
s1_output = perform_inference(s1_test_img, "Stage 1 Primitive")
print(s1_output)

# 2. Stage 2 Spotcheck: Boolean operations (Checking for Logical Anchoring).
print("\n" + "="*60)
print("STAGE 2 LOGIC CHECK (Boolean Operations)")
print("="*60)
s2_test_img = os.path.join(PROJECT_ROOT, 'dataset/stage2/images/csg_20005.png')
s2_output = perform_inference(s2_test_img, "Stage 2 CSG")
print(s2_output)

In [ ]:
# @title 3. Quantitative cross-check
def check_for_hallucinations(text):
    """Simple check to ensure Stage 1 hasn't started hallucinating min/max where they don't belong."""
    csg_ops = ["min(", "max(", "-sd"]
    found = [op for op in csg_ops if op in text]
    return found

print("Verifying structural integrity...")
s1_hallucinations = check_for_hallucinations(s1_output)
if len(s1_hallucinations) > 0:
    print(f"[WARNING] Stage 1 regression: Boolean logic leaking into single primitives: {s1_hallucinations}")
else:
    print("[PASS] Stage 1 remains focused on single primitives.")

s2_logic = check_for_hallucinations(s2_output)
if len(s2_logic) > 0:
    print(f"[PASS] Stage 2 logic anchoring confirmed: {s2_logic}")
else:
    print("[FAIL] Stage 2 logic missing Boolean operators.")

In [ ]:
# @title 4. Coordinate precision audit
import re

def extract_coords(text):
    """Finds the first vec2 or vec3 center coordinate in the shader code."""
    # Look for patterns like p - vec2(0.1, 0.2) or p - vec3(0.1, 0.2, 0.3)
    match = re.search(r'vec[23]\(([-\d.]+),\s*([-\d.]+)(?:,\s*([-\d.]+))?\)', text)
    if match:
        coords = [float(match.group(1)), float(match.group(2))]
        if match.group(3):
            coords.append(float(match.group(3)))
        return coords
    return None

def audit_spatial_drift(num_samples=5):
    """Calculates the Euclidean distance between predicted and ground truth coordinates."""
    dataset_path = os.path.join(PROJECT_ROOT, 'dataset/stage2/dataset.jsonl')

    with open(dataset_path, 'r') as f:
        data = [json.loads(line) for line in f][:num_samples]

    print("\n" + "="*60)
    print("SPATIAL PRECISION AUDIT (Pixel Drift)")
    print("="*60)

    for i, sample in enumerate(data):
        # 1. Run inference.
        raw_output = perform_inference(sample['image_path'])

        # 2. Extract coordinates.
        pred_coords = extract_coords(raw_output)
        # Assuming your ground truth is stored in the analysis/code or a separate field.
        # Here we extract from the 'code' field of the ground truth sample.
        gt_coords = extract_coords(sample['code'])

        if pred_coords and gt_coords:
            # Calculate Euclidean distance.
            dist = sum((p - g) ** 2 for p, g in zip(pred_coords, gt_coords)) ** 0.5
            status = "PRECISION PASS" if dist < 0.05 else "DRIFT DETECTED"
            print(f"Sample {i+1}: {status} | Error Distance: {dist:.4f}")
        else:
            print(f"Sample {i+1}: ERROR - Could not parse coordinates for comparison.")

# Run the audit on 5 samples.
audit_spatial_drift(num_samples=5)